# Landscape metrics

In [1]:
import matplotlib as mpl
import numpy as np
import seaborn as sns
import os
import pandas as pd
import swisslandstats as sls
import pylandstats as pls

Module c:\Users\VICTUS\anaconda3\envs\osgeo-env-v1\lib\site-packages\pylandstats\landscape.py has not been compiled for Transonic-Numba


## Compute class metrics

In [2]:
RASTERIZED_DIR = "./data/conflict/enviro/nodissolved/rasterized"

tif_files = sorted([
    os.path.join(RASTERIZED_DIR, f)
    for f in os.listdir(RASTERIZED_DIR)
    if f.endswith(".tif")
])

print(f"Found {len(tif_files)} file(s) to process:")
for f in tif_files:
    print(" ", os.path.basename(f))

Found 9 file(s) to process:
  ara_lcvoc_2000.tif
  ara_lcvoc_2010.tif
  ara_lcvoc_2022.tif
  casa_lcvoc_2000.tif
  casa_lcvoc_2010.tif
  casa_lcvoc_2022.tif
  meta_lcvoc_2000.tif
  meta_lcvoc_2010.tif
  meta_lcvoc_2022.tif


In [ ]:
results = []
for tif_path in tif_files:
    filename = os.path.splitext(os.path.basename(tif_path))[0]
    region, _, year = filename.split("_")[:3]
    
    print(f"Processing: {filename}...")
    ls = pls.Landscape(tif_path, res=(30, 30), nodata=0)
    
    # Diagnostics
    print("Classes:", ls.classes)
    print("Resolution:", ls.cell_width, ls.cell_height)
    print("Shape:", ls.landscape_arr.shape)
    for cls in ls.classes:
        count = np.sum(ls.landscape_arr == cls)
        print(f"  Class {cls}: {count:,} pixels")

    df = ls.compute_class_metrics_df(metrics=[
        'total_area',
        'proportion_of_landscape',
        'number_of_patches',
        'patch_density',
        'largest_patch_index',
        'edge_density',
        'landscape_shape_index',
    ])
    
    df["region"] = region
    df["year"]   = year
    results.append(df)

In [ ]:
# Combine all results
all_class_df = pd.concat(results)
# Reset index so 'class' becomes a column
all_class_df2 = all_class_df.reset_index()
# Map class values to labels
all_class_df2['class_val'] = all_class_df2['class_val'].map({1: 'No Conflict', 2: 'Moderate', 3: 'High'})
all_class_df2

In [ ]:
all_class_df2.to_csv("./data/landscape metrics/all_classmetrics.csv")